### Imports

In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### download all tables from wiki and select the correct one

In [2]:
df_list=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=df_list[0]
df.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### remove all not assigned boroughs

In [3]:
df=df.loc[df['Borough'] !='Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df['Borough'].values

array(['North York', 'North York', 'Downtown Toronto', 'North York',
       'Downtown Toronto', 'Etobicoke', 'Scarborough', 'North York',
       'East York', 'Downtown Toronto', 'North York', 'Etobicoke',
       'Scarborough', 'North York', 'East York', 'Downtown Toronto',
       'York', 'Etobicoke', 'Scarborough', 'East Toronto',
       'Downtown Toronto', 'York', 'Scarborough', 'East York',
       'Downtown Toronto', 'Downtown Toronto', 'Scarborough',
       'North York', 'North York', 'East York', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East York', 'Downtown Toronto', 'West Toronto', 'Scarborough',
       'North York', 'North York', 'East Toronto', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East Toronto', 'Downtown Toronto', 'North York', 'North York',
       'Scarborough', 'North York', 'North York', 'East Toronto',
       'North York', 'York', 'North York', 'Scarborough', 'Nort

### Make sure that postal codes appear only one

In [5]:
df['Postal Code'].value_counts(ascending=False)

M5L    1
M3B    1
M4H    1
M6S    1
M2P    1
      ..
M2L    1
M3A    1
M1W    1
M6B    1
M9B    1
Name: Postal Code, Length: 103, dtype: int64

### Shape of dataframe

In [6]:
df.shape

(103, 3)

### import coordinates csv
geocoders didnt work

In [7]:
gc_df=pd.read_csv('gc.csv')
gc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### inner join both dataframes

In [8]:
mi_df = pd.merge(left=df, right=gc_df, left_on='Postal Code', right_on='Postal Code')
mi_df.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Use geopy library to get the latitude and longitude values of Toronto.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mi_df['Latitude'], mi_df['Longitude'], mi_df['Borough'], mi_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'CQHEIJ0DPLLRPNGAACTH0I1RGMNVES3IX3I3X0XNZHAZ3BJU' # your Foursquare ID
CLIENT_SECRET = 'G2IBTFHRKA2G4ZJYVSZ1SXWSY5TSSYTAOUTNSGV5FLMQMFEE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CQHEIJ0DPLLRPNGAACTH0I1RGMNVES3IX3I3X0XNZHAZ3BJU
CLIENT_SECRET:G2IBTFHRKA2G4ZJYVSZ1SXWSY5TSSYTAOUTNSGV5FLMQMFEE


#### a function to repeat the process to get information for all the neighborhoods in Toronto

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues=mi_df

In [14]:
toronto_venues=getNearbyVenues(mi_df['Neighbourhood'],mi_df['Latitude'],mi_df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Let's check the size of the resulting dataframe

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(2167, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [16]:
# filder for resturans only
toronto_res=toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Restaurant')]
toronto_res.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
11,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
29,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
35,"Regent Park, Harbourfront",43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant
55,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
67,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Nando's,43.661728,-79.386391,Portuguese Restaurant
68,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
73,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tokyo Sushi,43.665885,-79.386977,Sushi Restaurant
76,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Como En Casa,43.665160,-79.384796,Mexican Restaurant


In [17]:
toronto_res.shape

(483, 7)

In [18]:
# convert all columns to 1/0
toronto_res_cat = pd.get_dummies(toronto_res[['Venue Category']], prefix="", prefix_sep="")
toronto_res_cat['Neighborhood'] = toronto_res['Neighborhood'] 

fixed_columns = [toronto_res_cat.columns[-1]] + list(toronto_res_cat.columns[:-1])
toronto_res_cat = toronto_res_cat[fixed_columns]

toronto_res_cat.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_res_grouped = toronto_res_cat.groupby('Neighborhood').mean().reset_index()
toronto_res_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,...,0.000000,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.50,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.000000,0.000000
4,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.153846,0.0,0.0,0.076923,0.0,0.076923,0.0,0.0,0.076923,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Victoria Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
57,Westmount,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.00,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
58,"Wexford, Maryvale",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
59,"Willowdale, Willowdale East",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.0,0.0,0.181818,0.0,0.000000,0.0,0.0,0.000000,0.090909


In [20]:
# clusster the data
kclusters = 5

toronto_res_grouped_clustering = toronto_res_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_res_grouped_clustering)

kmeans.labels_[0:10] 

array([4, 4, 0, 4, 4, 4, 4, 4, 1, 4])

In [21]:
# combine with other important columns
toronto_res_grouped_new=toronto_res_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_res_grouped

toronto_merged = mi_df

toronto_merged = toronto_merged.join(toronto_res_grouped.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
toronto_res_grouped

,Cluster Labels,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,4,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,4,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,...,0.000000,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,0,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.50,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,4,"Bedford Park, Lawrence Manor East",0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.000000,0.000000
4,4,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.153846,0.0,0.0,0.076923,0.0,0.076923,0.0,0.0,0.076923,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,4,Victoria Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
57,0,Westmount,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.00,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
58,3,"Wexford, Maryvale",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
59,4,"Willowdale, Willowdale East",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.181818,0.0,0.000000,0.0,0.0,0.000000,0.090909


In [23]:
toronto_merged=toronto_merged.fillna(0)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Now we will analyse which clusster has the least competition for resturants

In [24]:

df1=toronto_merged
df1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.000000
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4.0,0.041667,0.041667,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.0,0.0,0.041667,0.0,0.0,0.041667
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [25]:
# group by the cluster and sum all the inputs
df2=df1.groupby('Cluster Labels').sum()
df2

,Latitude,Longitude,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0.0,1923.429081,-3494.456319,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1.0,87.541116,-158.726738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000
2.0,174.974469,-317.622178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3.0,131.091786,-238.143787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4.0,2184.538144,-3968.957728,0.041667,2.951778,0.784106,0.089286,0.077961,0.142857,0.478709,1.852106,...,1.146508,0.0625,0.076923,3.384034,0.076923,1.585209,0.041667,0.5,0.838144,1.948052


In [26]:
#dropping the not required columns
df2=df2.drop(columns=['Latitude', 'Longitude'])
df2

,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000
2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4.0,0.041667,2.951778,0.784106,0.089286,0.077961,0.142857,0.478709,1.852106,0.077961,0.590632,...,1.146508,0.0625,0.076923,3.384034,0.076923,1.585209,0.041667,0.5,0.838144,1.948052


In [27]:
# sum in axis 1 will give as the density for each cluster
df2.sum(axis=1)

Cluster Labels
0.0     3.0
1.0     2.0
2.0     4.0
3.0     3.0
4.0    50.0
dtype: float64

#### no we will find the most recommended resturants for each area

In [28]:
# remove clustter with the highest density
df_no_class_4=df1.loc[df1['Cluster Labels']!=4.0]
df_no_class_4=df_no_class_4.drop(columns=['Latitude', 'Longitude','Postal Code','Borough','Cluster Labels'])

In [29]:
df_no_class_4.describe()

,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,...,Seafood Restaurant,Southern / Soul Food Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
count,53.0,53.0,53.0,53.0,53.0,53.0,53.000000,53.000000,53.0,53.0,...,53.0,53.0,53.0,53.0,53.0,53.000000,53.0,53.0,53.0,53.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.025157,0.033019,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.006289,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.143953,0.155336,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.045787,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0


In [30]:
# Loop through all the Neighbourhood to find each resturants will be more competitive
for num,i in df_no_class_4.iterrows():
    l=[]
    for j in i.keys():
        if i[j]==0.0:
            l.append(j)
    print(f'In this Neighbourhood : {i[0]} has the lowest competition for this types for restaurants {l} ')

In this Neighbourhood : Parkwoods has the lowest competition for this types for restaurants ['Afghan Restaurant', 'American Restaurant', 'Asian Restaurant', 'Belgian Restaurant', 'Brazilian Restaurant', 'Cajun / Creole Restaurant', 'Caribbean Restaurant', 'Chinese Restaurant', 'Colombian Restaurant', 'Comfort Food Restaurant', 'Cuban Restaurant', 'Dim Sum Restaurant', 'Doner Restaurant', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Ethiopian Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'French Restaurant', 'German Restaurant', 'Gluten-free Restaurant', 'Greek Restaurant', 'Hakka Restaurant', 'Indian Restaurant', 'Italian Restaurant', 'Japanese Restaurant', 'Korean BBQ Restaurant', 'Korean Restaurant', 'Latin American Restaurant', 'Malay Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Modern European Restaurant', 'Molecular Gastronomy Restaurant', 'Moroccan Restaurant', 'New American Restaurant',